# Fit the DDM on individual data

In [1]:
import pandas as pd

from rlssm.model.models_DDM import DDModel
from rlssm.utility.load_data import load_example_dataset

## Import the data

In [2]:
data = load_example_dataset(hierarchical_levels = 1)

data.head()

,participant,block_label,trial_block,f_cor,f_inc,cor_option,inc_option,times_seen,rt,accuracy
0,11,1,1,59,44,4,3,1,1.462756,1
1,11,1,2,53,44,4,2,1,1.256185,0
2,11,1,3,64,55,4,3,2,1.264878,1
3,11,1,4,47,36,3,1,2,1.399706,0
4,11,1,5,54,55,4,3,4,1.381221,1


## Initialize the model

In [3]:
model = DDModel(hierarchical_levels = 1)

Using cached StanModel


## Fit

In [4]:
# sampling parameters
n_warmup = 500
n_sampling = 500
n_chains = 2
n_thin = 1

In [5]:
model_fit = model.fit(
    data,
    thin = n_thin,
    iter_warmup=n_warmup,
    iter_sampling=n_sampling,
    chains = n_chains,
    pointwise_waic=False)

20:29:24 - cmdstanpy - INFO - CmdStan start processing


Fitting the model using the priors:
drift_priors {'mu': 1, 'sd': 5}
threshold_priors {'mu': 0, 'sd': 5}
ndt_priors {'mu': 0, 'sd': 5}


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

20:29:24 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:

Command and output files:
RunSet: chains=2, chain_ids=[1, 2], num_processes=2
 cmd (chain 1):
	['/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/DDM/DDM', 'id=1', 'random', 'seed=12550', 'data', 'file=/tmp/tmp20xkznp1/cvfh83yx.json', 'output', 'file=/tmp/tmp20xkznp1/DDMsfshqvhl/DDM-20220705202924_1.csv', 'method=sample', 'num_samples=500', 'num_warmup=500', 'thin=1', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[-1, -1]
 per-chain output files (showing chain 1 only):
 csv_file:
	/tmp/tmp20xkznp1/DDMsfshqvhl/DDM-20220705202924_1.csv
 console_msgs (if any):
	/tmp/tmp20xkznp1/DDMsfshqvhl/DDM-20220705202924_0-stdout.txt
Consider re-running with show_console=True if the above output is unclear!

### get Rhat

In [ ]:
model_fit.rhat

### get wAIC

In [ ]:
model_fit.waic

## Posteriors

In [ ]:
model_fit.samples.describe()

In [ ]:
import seaborn as sns
sns.set(context = "talk", 
        style = "white", 
        palette = "husl",
        rc={'figure.figsize':(15, 8)})

In [ ]:
model_fit.plot_posteriors(height=5, show_intervals="HDI", alpha_intervals=.05);

## Posterior predictives

### Ungrouped

In [ ]:
pp = model_fit.get_posterior_predictives_df(n_posterior_predictives=100)
pp

In [ ]:
pp_summary = model_fit.get_posterior_predictives_summary(n_posterior_predictives=100)
pp_summary

In [ ]:
model_fit.plot_mean_posterior_predictives(n_posterior_predictives=100, figsize=(20,8), show_intervals='HDI');

In [ ]:
model_fit.plot_quantiles_posterior_predictives(n_posterior_predictives=100, kind='shades');

### Grouped

In [ ]:
import numpy as np

In [ ]:
# Define new grouping variables, in this case, for the different choice pairs, but any grouping var can do
data['choice_pair'] = 'AB'
data.loc[(data.cor_option == 3) & (data.inc_option == 1), 'choice_pair'] = 'AC'
data.loc[(data.cor_option == 4) & (data.inc_option == 2), 'choice_pair'] = 'BD'
data.loc[(data.cor_option == 4) & (data.inc_option == 3), 'choice_pair'] = 'CD'

data['block_bins'] = pd.cut(data.trial_block, 8, labels=np.arange(1, 9))

In [ ]:
model_fit.get_grouped_posterior_predictives_summary(
                grouping_vars=['block_label', 'choice_pair'],
                quantiles=[.3, .5, .7],
                n_posterior_predictives=100)

In [ ]:
model_fit.get_grouped_posterior_predictives_summary(
                grouping_vars=['block_bins'],
                quantiles=[.3, .5, .7],
                n_posterior_predictives=100)

In [ ]:
model_fit.plot_mean_grouped_posterior_predictives(grouping_vars=['block_bins'], 
                                                  n_posterior_predictives=100, 
                                                  figsize=(20,8));

In [ ]:
model_fit.plot_quantiles_grouped_posterior_predictives(
    n_posterior_predictives=100,
    grouping_var='choice_pair',
    kind='shades',
    quantiles=[.1, .3, .5, .7, .9]);